In [ ]:
# ⚡ 更快版 ResNet18 訓練：啟動提示 + 小圖訓練 + 記憶曲線

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 設定裝置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 設定資料與模型儲存路徑
data_path = "C:/Users/echo6/Downloads/Food/data/food-101-tiny"
model_save_path = "resnet18_food.pth"

# 減小圖片尺寸，加速訓練（128x128）
transform_train = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

transform_val = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# 資料載入
train_dataset = datasets.ImageFolder(root=os.path.join(data_path, "train"), transform=transform_train)
val_dataset = datasets.ImageFolder(root=os.path.join(data_path, "valid"), transform=transform_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 建立 ResNet18 模型
from torchvision.models import ResNet18_Weights
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))
model = model.to(device)

# 設定損失與優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# 訓練與驗證函數
def train_one_epoch():
    model.train()
    running_loss, correct, total = 0, 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return running_loss / len(train_loader), correct / total

def evaluate():
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return correct / total

# 執行訓練（帶有提示）
train_acc_list, val_acc_list = [], []
EPOCHS = 5  # 可以調小輪數，加快初步測試
for epoch in range(EPOCHS):
    print(f"🚀 開始訓練第 {epoch+1} 輪...")
    loss, train_acc = train_one_epoch()
    val_acc = evaluate()
    train_acc_list.append(train_acc)
    val_acc_list.append(val_acc)
    print(f"✅ 完成第 {epoch+1} 輪：Loss={loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")

# 儲存模型
torch.save(model.state_dict(), model_save_path)
print("✅ ResNet18 模型已儲存為 resnet18_food.pth")

# 顯示準確率曲線
plt.plot(train_acc_list, label="Train Acc")
plt.plot(val_acc_list, label="Val Acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("ResNet18 訓練準確率趨勢")
plt.legend()
plt.grid(True)
plt.show()


🚀 開始訓練第 1 輪...
✅ 完成第 1 輪：Loss=1.4680, Train Acc=0.5087, Val Acc=0.5860
🚀 開始訓練第 2 輪...
✅ 完成第 2 輪：Loss=0.7463, Train Acc=0.7513, Val Acc=0.6400
🚀 開始訓練第 3 輪...
✅ 完成第 3 輪：Loss=0.5346, Train Acc=0.8287, Val Acc=0.6440
🚀 開始訓練第 4 輪...
✅ 完成第 4 輪：Loss=0.4086, Train Acc=0.8633, Val Acc=0.6840
🚀 開始訓練第 5 輪...
✅ 完成第 5 輪：Loss=0.3389, Train Acc=0.8933, Val Acc=0.6680
✅ ResNet18 模型已儲存為 resnet18_food.pth
